In [0]:
# Order data
df = spark.read.csv('/Volumes/ecommerce/transactionsdb/ingestion/orders/', header=True)
display(df.limit(5))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [0]:
# Order items data
df = spark.read.csv('/Volumes/ecommerce/transactionsdb/ingestion/order_items/', header=True)
display(df.limit(5))

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [0]:
# customer data
df = spark.read.csv('/Volumes/ecommerce/transactionsdb/ingestion/customers/', header=True)
display(df.limit(5))

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [0]:
df.select('customer_id').distinct().count(), df.select('customer_unique_id').distinct().count()

(99441, 96096)

In [0]:
%sql

select current_schema();

current_schema()
default


In [0]:
%sql

-- select count(*) from ecommerce.transactionsdb.order_items_fact; 108628
-- select count(distinct customer_id) from ecommerce.transactionsdb.order_items_fact; 95372

-- Top 2 orders for EACH customer
with order_totals 
  as (
    select 
      order_id,
      customer_unique_id,
      sum(price) as total_order_price,
      sum(freight_value) as total_order_freight
    from ecommerce.transactionsdb.order_items_fact
    group by order_id, customer_unique_id
  ),
  ranked_orders 
    as (
      select 
        *,
        row_number() over(partition by customer_unique_id order by total_order_price desc) as order_rank
      from order_totals
  ),
  top_two_orders
    as (
      select 
        ro.order_id, ro.customer_unique_id,
        cd.customer_zip_code_prefix as customer_zipcode,
        cd.customer_city as customer_city,
        cd.customer_state as customer_state,
        ro.total_order_price as order_price,
        ro.total_order_freight as order_freight_value,
        ro.order_rank
      from ranked_orders ro
          inner join ecommerce.transactionsdb.customers_dim cd
          on ro.customer_unique_id = cd.customer_unique_id
      where ro.order_rank <= 2 
        and cd.`__END_AT` is null
    )
select * from top_two_orders
order by customer_unique_id, order_rank
limit 10;




order_id,customer_unique_id,customer_zipcode,customer_city,customer_state,order_price,order_freight_value,order_rank
e22acc9c116caa3f2b7121bbb380d08e,0000366f3b9a7992bf8c76cfdf3221e2,7787,cajamar,SP,129.90,12.00,1
3594e05a005ac4d06a72673270ef9ec9,0000b849f77a49e4a4ce2b2a4ca5be3f,6053,osasco,SP,18.90,8.29,1
b33ec3b699337181488304f362a6b734,0000f46a3911fa3c0805444483337064,88115,sao jose,SC,69.00,17.22,1
41272756ecddd9a9ed0180413cc22fb6,0000f6ccb0745a6a4b88665a16c9f078,66812,belem,PA,25.99,17.63,1
d957021f1127559cd947b62533f484f7,0004aac84e0df4da2b147fca70cf8255,18040,sorocaba,SP,180.00,16.89,1
3e470077b690ea3e3d501cffb5e0c499,0004bd2a26a76fe21f786e4fbd80607f,5036,sao paulo,SP,154.00,12.98,1
d0028facea13f508e880202d7097a5a1,00050ab1314c0e55a6ca13cf7181fecf,13084,campinas,SP,27.99,7.39,1
44e608f2db00c74a1fe329de44416a4e,00053a61a98854899e70ed204dd4bafe,80410,curitiba,PR,382.00,37.18,1
ae76bef74b97bcb0b3e355e60d9a6f9c,0005e1862207bf6ccc02e4228effd9a0,25966,teresopolis,RJ,135.00,15.12,1
01b330808c5819a6a3cb79b72f0b8288,0005ef4cd20d2893f0d9fbd94d3c0d97,65060,sao luis,MA,104.90,24.86,1


In [0]:
%sql

select 
  * 
from ecommerce.transactionsdb.top_two_orders_for_each_customer_view
limit 10;

order_id,customer_unique_id,customer_zipcode,customer_city,customer_state,order_price,order_freight_value,order_rank
f7fa5cf8386e51037856df1add3e1228,000a5ad9c4601d2bbdd9ed765d5213b3,90560,porto alegre,RS,76.99,14.29,1
1fc81b1074b61a259529663946b10a32,0010fb34b966d44409382af9e8fd5b77,4321,sao paulo,SP,49.95,11.85,1
3eb5983ab1e64ba4a4dd7425a6d6d48b,0014a5a58da615f7b01a4f5e194bf5ea,1407,sao paulo,SP,88.00,11.82,1
1d88068495a8986f241f769664ac98de,00196c4c9a3af7dd2ad10eade69c926f,13287,vinhedo,SP,22.32,15.26,1
18bb4acb90913593d37049c601660009,0019e8c501c85848ac0966d45226fa1d,78015,cuiaba,MT,159.99,37.28,1
a59ef0abffbef8ddaae23600b6ee6604,001ae44fa04911a9e9577356dce6c63c,9581,sao caetano do sul,SP,99.90,16.33,1
4c5fb984844ad48dc34f8acf42eb0eda,002043098f10ba39a4600b6c52fbfe3c,49015,aracaju,SE,176.99,60.82,1
9256fda5584ec43f6212f63bdf17b2ec,002311514717ca8b65b09a26cdf7b91a,7114,guarulhos,SP,116.25,8.07,1
dc9b66f791e6bc0d3a44e2c513a1d117,0025795df7a7d077c4c90162fa820085,2323,sao paulo,SP,177.90,36.18,1
b4cafb8fabcef776ee4b1108769b47ae,00377a18050b898b93e2c774be9947db,78065,cuiaba,MT,49.99,15.79,1
